# Join nationalmaps demographic data, with latitude and longitude and .DAT sales data

In [24]:
from pathlib import Path
import geopandas as gpd
import pandas as pd


DATA_DIR = (Path().resolve().parent / "data" )
joined_data_path = DATA_DIR / "01_interim" / "nationalmaps" / "joined_nationalmaps.shp"
dat_with_lat_lon_path = DATA_DIR / "01_interim/valuergeneral/DAT_with_lat_lon.csv"

features_path = DATA_DIR / "02_processed" / "features_raw.csv"

In [3]:
df_natmaps = gpd.read_file(joined_data_path)
df_natmaps.shape

(2454, 71)

In [4]:
df_dat_with_lat_lon = pd.read_csv(dat_with_lat_lon_path)
df_dat_with_lat_lon.shape

(58258, 47)

### Demographic zones data mapped to addresses.

In [5]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# Assuming G is your GeoPandas dataframe with a 'geometry' column of Polygons
# and A is your Pandas dataframe with 'lat' and 'lon' columns.
A = df_dat_with_lat_lon.copy()
G = df_natmaps.copy()

# Convert the 'lat' and 'lon' columns of A into a GeoSeries of Point geometries
geometry = [Point(xy) for xy in zip(A['lon'], A['lat'])]
A = gpd.GeoDataFrame(A, geometry=geometry)

# Ensure both dataframes use the same coordinate reference system (CRS)
A.set_crs(G.crs, inplace=True)

# Perform the spatial join; this associates rows of A with polygons in G
result = gpd.sjoin(A, G, how='left', op='within')

# 'result' now has all columns from A and G where each point in A falls within a polygon in G

## Select variables to create feature set from

In [15]:
import itertools

chunk_size = 10
c = list(result.columns)
chunks = [c[i:i + chunk_size] for i in range(0, len(c), chunk_size)]
# use itertools to iterate over sequential chunks of size 10 in the list
for chunk in chunks:
    print(chunk)

['Unnamed: 0', 'index', 'fileType', 'districtCode', 'downloadDateTime', 'submitterUserId', 'districtCode.1', 'propertyId', 'saleCounter', 'downloadDateTime.1']
['propertyName', 'propertyUnitNumber', 'propertyHouseNumber', 'propertyStreetName', 'propertyLocality', 'propertyPostCode', 'area', 'areaType', 'contractDate', 'settlementDate']
['purchasePrice', 'zoning', 'natureOfProperty', 'primaryPurpose', 'strataLotNumber', 'componentCode', 'saleCode', 'percentInterestOfSale', 'dealingNumber', 'districtCode.2']
['propertyId.1', 'saleCounter.1', 'downloadDateTime.2', 'propertyLegalDescription', 'districtCode.3', 'propertyId.2', 'saleCounter.2', 'downloadDateTime.3', 'purchaserVendor', 'totalRecords']
['totalBRecords', 'totalCRecords', 'totalDRecords', 'address_str', 'address', 'lat', 'lon', 'geometry', 'index_right', 'STE_CODE21']
['STE_NAME21', 'GCC_CODE21', 'GCC_NAME21', 'SA4_CODE21', 'SA4_NAME21', 'SA3_CODE21', 'SA3_NAME21', 'SA2_CODE21', 'SA2_NAME21', 'Males']
['Females', 'Persons', 'Sex

In [26]:
df_joined = pd.DataFrame(result)

### Filter out unwanted columns

In [27]:
f_pop_age = lambda c: (c.startswith('P') or c.startswith('M') or c.startswith('F')) and ('_' in c) and c[1].isdigit() or ('Median_age' in c)
f_income = lambda c: 'OBS_VALUE' == c
f_pop = lambda c: c in ['Females', 'Males', 'Persons', 'Sex_ratio']
f_purchase_metadata = lambda c: c in ['settlementDate', 'natureOfProperty', 'primaryPurpose', 'propertyUnitNumber', 'propertyHouseNumber']
col_filters = [f_pop_age, 
               f_income, 
               f_pop, 
               f_purchase_metadata]
# new dataframe with only columns that match the filter
df_joined = df_joined[[c for f in col_filters for c in df_joined.columns if f(c)]]
df_joined.head()

,Median_age,M0_4,M5_9,M10_14,M15_19,M20_24,M25_29,M30_34,M35_39,M40_44,...,OBS_VALUE,Males,Females,Persons,Sex_ratio,propertyUnitNumber,propertyHouseNumber,settlementDate,natureOfProperty,primaryPurpose
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,20,59,20211223.0,R,RESIDENCE
1,41.5,636.0,710.0,774.0,696.0,618.0,408.0,522.0,610.0,722.0,...,16600,10048.0,10574.0,20622.0,95.0,NaN,28,20211223.0,R,RESIDENCE
2,43.4,349.0,430.0,457.0,414.0,303.0,234.0,284.0,344.0,403.0,...,9605,6038.0,5907.0,11945.0,102.2,NaN,808 C,20211223.0,V,VACANT LAND
3,33.4,429.0,453.0,376.0,500.0,1124.0,1037.0,720.0,571.0,459.0,...,14159,8607.0,8147.0,16754.0,105.6,NaN,51,20211223.0,R,RESIDENCE
4,38.6,338.0,333.0,331.0,310.0,284.0,219.0,272.0,308.0,301.0,...,7176,4606.0,4482.0,9088.0,102.8,NaN,16,20211224.0,R,RESIDENCE


In [28]:
df_joined.to_csv(features_path, index=False)

In [29]:
df_joined.info()

<class 'pandas.core.frame.DataFrame'>
Index: 58258 entries, 0 to 58257
Data columns (total 65 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Median_age           15196 non-null  float64
 1   M0_4                 15196 non-null  float64
 2   M5_9                 15196 non-null  float64
 3   M10_14               15196 non-null  float64
 4   M15_19               15196 non-null  float64
 5   M20_24               15196 non-null  float64
 6   M25_29               15196 non-null  float64
 7   M30_34               15196 non-null  float64
 8   M35_39               15196 non-null  float64
 9   M40_44               15196 non-null  float64
 10  M45_49               15196 non-null  float64
 11  M50_54               15196 non-null  float64
 12  M55_59               15196 non-null  float64
 13  M60_64               15196 non-null  float64
 14  M65_69               15196 non-null  float64
 15  M70_74               15196 non-null  floa